# Big Data Intelligence Assignment 2

This is the code for the Assignment 2. Some parts taking too much execution time on my PC are commented, you should uncomment them if you want to see the results.

## Experiment 1 : Data Preprocessing

First we are going to create a map for the user indices and then we are going to build the matrices as specified, the matrices are 0 indexed so X[user_1, movie_1] = X[0, 0].
We also wrote some tests to ensure we preprocessed the data correctly.

In [1]:
# Users Mapping

user_ids = []
print("Mapping Users...")
with open('./Project2-data/users.txt', 'r') as file:
    user_ids = [int(line.strip()) for line in file]
user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
print("Users Mapping Done")

Mapping Users...
Users Mapping Done


In [2]:
# Data Preprocessing

import numpy as np
from scipy.sparse import csr_matrix
from tqdm import tqdm

def preprocess_file(filename: str):
    """
    Preprocesses a Netflix-style dataset file into sparse matrices for ratings and indicators.

    Args:
        filename (str): Path to the dataset file.

    Returns:
        csr_matrix: Sparse matrix (X) containing user ratings.
        csr_matrix: Sparse matrix (IM) indicating known ratings (1) vs unknown ratings (0).
    """
    print(f"Preprocessing file {filename}...")
    num_users = len(user_id_to_index)
    num_movies = 10000
    
    X = np.zeros((num_users, num_movies), dtype=np.int8) # rating matrix
    IM = np.zeros((num_users, num_movies), dtype=np.int8) # indicator matrix
    
    with open(filename, 'r') as f:
        for line in tqdm(f):
            user_id, movie_idx, rating, _ = line.split()
            user_idx, movie_idx, rating = user_id_to_index[int(user_id)], int(movie_idx), int(rating)
            X[user_idx, movie_idx - 1] = rating # adjust movie index to 0-based
            IM[user_idx, movie_idx - 1] = 1
            
    print("Data preprocessing complete.")
    print(f"Matrix X shape: {X.shape}")
    print(f"Known ratings count: {int(IM.sum())}")
    
    return csr_matrix(X), csr_matrix(IM)

X_train, IM_train = preprocess_file('./Project2-data/netflix_train.txt')
X_test, IM_test = preprocess_file('./Project2-data/netflix_test.txt')

Preprocessing file ./Project2-data/netflix_train.txt...


6897746it [00:10, 678705.54it/s]


Data preprocessing complete.
Matrix X shape: (10000, 10000)
Known ratings count: 6897746
Preprocessing file ./Project2-data/netflix_test.txt...


1719466it [00:02, 679243.58it/s]


Data preprocessing complete.
Matrix X shape: (10000, 10000)
Known ratings count: 1719466


In [3]:
# Testing
user_id, movie_idx, rating, _ = open('./Project2-data/netflix_train.txt', 'r').readline().split()
assert X_train[user_id_to_index[int(user_id)], int(movie_idx) - 1] == int(rating)

user_id, movie_idx, rating, _ = open('./Project2-data/netflix_test.txt', 'r').readline().split()
assert X_test[user_id_to_index[int(user_id)], int(movie_idx) - 1] == int(rating)

assert (X_train.data >= 1).all() and (X_train.data <= 5).all()

assert (X_test.data >= 1).all() and (X_test.data <= 5).all()

print("All tests passed.")

All tests passed.


## Experiment 2 : Collaborative Filtering

Here we are going to compute the similarities first, then implement the user-based collaborative filtering. After that we will check the RMSE and execution time for different values of k.

In [4]:
# Similarity Matrix
import numpy as np

def compute_similarity_matrix(X):
    """
    Computes the similarity matrix between users based on the cosine similarity.

    Args:
        X (csr_matrix): Sparse matrix containing user ratings.

    Returns:
        np.ndarray: Similarity matrix between users.
    """
    print("Computing similarity matrix...")
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    norm_X = X / norms
    S = norm_X @ norm_X.T
    print("Similarity matrix computed.")
    
    return S

S_train = compute_similarity_matrix(X_train.toarray())

Computing similarity matrix...
Similarity matrix computed.


In [5]:
# Collaborative Filtering
import numpy as np

def user_based_collaborative_filtering(X_train, IM_train, X_test, IM_test, S_train, k_neighbors=500):
    """
    Args:
        X_train (np.ndarray): ratings matrix for training set
        IM_train (np.ndarray): indicator matrix for training set
        X_test (np.ndarray): ratings matrix for test set
        IM_test (np.ndarray): indicator matrix for test set
        S_train (np.ndarray): similarity matrix between users
        k_neighbors (int, optional): Defaults to 1000.

    Returns:
        rmse (float): root mean squared error of the predictions
        predictions (np.ndarray): predicted ratings for the test set
    """
    print("k_neighbors: ", k_neighbors)
    for i in range(X_train.shape[0]):
        top_k_indices = list(np.argsort(S_train[i, :])[-k_neighbors-1:]) # actually top k+1 indices since the self similarity is 1
        top_k_indices.pop() # remove self
        mask = np.ones(X_train.shape[0], dtype=bool)
        mask[top_k_indices] = False
        S_train[i, mask] = 0
    
    print("Computing predictions...") 
    weighted_sum = S_train @ X_train
    sim_sum = S_train @ IM_train
    sim_sum[sim_sum == 0] = 1 # avoid division by zero
    predictions = weighted_sum / sim_sum
    
    print("Computing RMSE...")
    actual_ratings = X_test[IM_test > 0]
    predicted_ratings = predictions[IM_test > 0]
    rmse = np.sqrt(np.mean((actual_ratings - predicted_ratings) ** 2))

    return rmse, predictions

print("Computing user-based collaborative filtering predictions...")
rmse, predictions = user_based_collaborative_filtering(X_train.toarray(), IM_train.toarray(), X_test.toarray(), IM_test.toarray(), S_train.copy())
print("Prediction shape : ", predictions.shape)
print(f"Final RMSE: {rmse}")
print("User-based collaborative filtering predictions computed.")

Computing user-based collaborative filtering predictions...
k_neighbors:  500
Computing predictions...
Computing RMSE...
Prediction shape :  (10000, 10000)
Final RMSE: 0.9791010971537276
User-based collaborative filtering predictions computed.


In [6]:
# Analysis

import matplotlib.pyplot as plt

def plot_rmse_vs_k():
    """
    Plots RMSE vs k for user-based collaborative filtering.
    """
    k_values = [1, 10, 100, 500, 1000, 2000, 5000, 10000]
    results = []
    for k in k_values:
        rmse, _ = user_based_collaborative_filtering(X_train.toarray(), IM_train.toarray(), X_test.toarray(), IM_test.toarray(), S_train.copy(), k_neighbors=k)
        results.append(rmse)

    # Plotting
    plt.plot(k_values, results)
    plt.xlabel("k")
    plt.ylabel("RMSE")
    plt.title("RMSE vs k")
    plt.xscale('log')
    plt.show()

# Uncomment to plot RMSE vs k. around 10 minutes to run for all the values of k.
# plot_rmse_vs_k() 

## Experiment 3 : Matrix Decomposition

In this part we are going to implement a Adam optimizer for matrix decomposition. We will also to focus a bit on initialization and parameters selection.

In [7]:
import numpy as np
import time
from tqdm import tqdm

def init_matrix(m, k, init_type):
    """
    Args:
        m (int): Number of rows.
        k (int): Number of columns.
        init_type (str): Type of initialization.
    
    Returns:
        np.ndarray: Initialized matrix
    """
    
    def uniform(m, k):
        return np.random.uniform(size=(m, k))
    
    def normal(m, k):
        return np.random.normal(0, 1/k, size=(m, k))
    
    def ones(m, k):
        return np.ones((m, k))
    
    def sparse(m, k):
        num_elements_to_fill = m*k // 100
    
        matrix = np.zeros((m, k))

        for _ in range(num_elements_to_fill):
            i, j = np.random.randint(0, m), np.random.randint(0, k)
            matrix[i, j] = np.random.randint(1, 6)
        
        return matrix
    
    matrix_init = {
        'uniform': uniform,
        'normal': normal,
        'ones': ones,
        'sparse': sparse
    }
    
    return matrix_init[init_type](m, k)

def adam_for_matrix_decomposition(X_train, IM_train, X_test, IM_test, k=50, alpha=1e-3, beta1=0.9, beta2=0.999, epsilon=1e-8, lambda_param=0.01, max_iter=1000, threshold=0.1, init_type='normal'):
    """
        Adam optimizer for matrix decomposition.
    """
    m, n = X_train.shape
    U = init_matrix(m, k, init_type)
    V = init_matrix(n, k, init_type)
    
    # Initialize Adam parameters
    m_U = np.zeros_like(U)
    v_U = np.zeros_like(U)
    m_V = np.zeros_like(V)
    v_V = np.zeros_like(V)
    
    X_pred = U @ V.T
    
    history = []
    init_time = time.time()
    for t in tqdm(range(1, max_iter + 1)):
        error = IM_train * (X_pred - X_train)
        grad_U = (error @ V) + 2 * lambda_param * U
        grad_V = (error.T @ U) + 2 * lambda_param * V
        
        m_U = beta1 * m_U + (1 - beta1) * grad_U
        v_U = beta2 * v_U + (1 - beta2) * (grad_U ** 2)
        m_U_hat = m_U / (1 - beta1 ** t)
        v_U_hat = v_U / (1 - beta2 ** t)
        U -= alpha * m_U_hat / (np.sqrt(v_U_hat) + epsilon)
        
        m_V = beta1 * m_V + (1 - beta1) * grad_V
        v_V = beta2 * v_V + (1 - beta2) * (grad_V ** 2)
        m_V_hat = m_V / (1 - beta1 ** t)
        v_V_hat = v_V / (1 - beta2 ** t)
        V -= alpha * m_V_hat / (np.sqrt(v_V_hat) + epsilon)
        
        X_pred = U @ V.T
        
        target = lambda_param * (np.linalg.norm(U) ** 2 + np.linalg.norm(V) ** 2)
        target += 0.5 * np.linalg.norm(IM_train * (X_train - X_pred)) ** 2
        
        rmse = np.sqrt(np.mean((X_test[IM_test > 0] - X_pred[IM_test > 0]) ** 2))
        history.append((target, rmse, time.time() - init_time))
        
        if t % 50 == 0:
            print(f"Iteration {t}, target: {target}", "RMSE: ", rmse)
        
        if t > 1 and abs(history[-1][0] - history[-2][0]) < threshold:
            print(f"Converged after {t} iterations.")
            break
    
    print(f"Adam completed in {time.time() - init_time} seconds after {t} iterations", "Final RMSE: ", rmse)
    
    return U, V, history

import matplotlib.pyplot as plt

def plot_target_rmse(k, lambda_param, init_type='normal', max_iter=1000):
    """
    Plots target function and RMSE over iterations.
    """
    _, _, history = adam_for_matrix_decomposition(X_train=X_train.toarray(), X_test=X_test.toarray(), IM_train=IM_train.toarray(), IM_test=IM_test.toarray(), k=k, lambda_param=lambda_param, init_type=init_type, max_iter=max_iter)
    targets = [target for target, _, _ in history]
    rmses = [rmse for _, rmse, _ in history]
    iterations = range(len(history))
    
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    ax1.plot(iterations, targets, c='red')
    ax1.set_ylabel('Target function')
    ax2.plot(iterations, rmses, c='blue')
    ax2.set_ylabel('RMSE')
    fig.suptitle('Target function and RMSE over iterations') 
    plt.show()
    
    return history # return history for testing

In [8]:
# Plot target function and RMSE for k=50 and lambda=0.01
# Uncomment to plot target function and RMSE over iterations. takes around 30 minutes to run.
#history1 = plot_target_rmse(50, 0.01)

In [9]:
# Different initializations
import matplotlib.pyplot as plt

def plot_final_rmse_vs_init_type():
    """
    Plots RMSE vs iterations for matrix decomposition.
    """
    init_types = ['uniform', 'normal', 'ones', 'sparse']
    results = []
    for init_type in init_types:
        _, _, history = adam_for_matrix_decomposition(X_train=X_train.toarray(), X_test=X_test.toarray(), IM_train=IM_train.toarray(), IM_test=IM_test.toarray(), init_type=init_type, max_iter=100)
        results.append(history[-1][1])

    # Plotting
    plt.bar(init_types, results, width=0.4)
    plt.xlabel("Initialization Type")
    plt.ylabel("RMSE")
    plt.title("RMSE vs Initialization type (max_iter=100)")
    plt.show()
    
# Uncomment to plot RMSE vs initialization type for matrix decomposition. takes around 15 minutes to run.
#plot_final_rmse_vs_init_type()

In [10]:
# Different Values of K
import matplotlib.pyplot as plt

def plot_final_rmse_vs_k_matrix_decomposition():
    k_values = [1, 10, 50, 100, 500, 1000, 2000]
    results = []
    for k in k_values:
        _, _, history = adam_for_matrix_decomposition(X_train=X_train.toarray(), X_test=X_test.toarray(), IM_train=IM_train.toarray(), IM_test=IM_test.toarray(), k=k, max_iter=100)
        results.append(history[-1][1])
    
    # Plotting
    plt.bar([str(el) for el in k_values], results, width=0.4)
    plt.xlabel("k")
    plt.ylabel("RMSE")
    plt.title("RMSE vs k (max_iter=100; init_type=normal)")
    plt.show()
    
    return results # return results for testing
    
# Uncomment to plot RMSE vs k for matrix decomposition. Takes around 70 minutes to run.
#results = plot_final_rmse_vs_k_matrix_decomposition()

In [11]:
# Different Values of Lambda
import matplotlib.pyplot as plt

def plot_final_rmse_vs_lambda_matrix_decomposition():
    lambda_values = [0.0001, 0.001, 0.01, 0.1]
    results = []
    for lambda_param in lambda_values:
        _, _, history = adam_for_matrix_decomposition(X_train=X_train.toarray(), X_test=X_test.toarray(), IM_train=IM_train.toarray(), IM_test=IM_test.toarray(), lambda_param=lambda_param, max_iter=100)
        results.append(history[-1][1])
    
    # Plotting
    plt.bar([str(el) for el in lambda_values], results, width=0.4)
    plt.xlabel("Lambda")
    plt.ylabel("RMSE")
    plt.title("RMSE vs Lambda (max_iter=100, init_type=normal)")
    plt.show()
    
    return results # return results for testing
    
# Uncomment to plot RMSE vs lambda for matrix decomposition. Takes around 17 minutes to run.
#results = plot_final_rmse_vs_lambda_matrix_decomposition()

In [12]:
# After parameters selection, we can use the best parameters to compute the final RMSE
# Uncomment to compute final RMSE with the best parameters. Takes around 30 minutes to run.
# history_n = plot_target_rmse(k=500, lambda_param=0.01, init_type='normal', max_iter=500)

## Comparison

Here we want to compare the performances of the 2 aproaches

In [13]:
# Execution time vs RMSE
import time
import matplotlib.pyplot as plt

def plot_execution_time_vs_rmse():
    """
    Plots execution time vs RMSE for user-based collaborative filtering and matrix decomposition.
    """
    # User-based collaborative filtering
    k_values = [1, 10, 100, 500, 1000]
    results1 = []
    for k in k_values:
        t = time.time()
        rmse, _ = user_based_collaborative_filtering(X_train.toarray(), IM_train.toarray(), X_test.toarray(), IM_test.toarray(), S_train.copy(), k_neighbors=k)
        results1.append((rmse, time.time() - t))
    
    # Matrix decomposition
    _, _, history = adam_for_matrix_decomposition(X_train=X_train.toarray(), X_test=X_test.toarray(), IM_train=IM_train.toarray(), IM_test=IM_test.toarray(), max_iter=100, k=500)
    results2 = [(history[i][1], history[i][2]) for i in range(0, len(history), 5)]
    
    # Plotting
    plt.figure(figsize=(10, 5))
    plt.plot(*zip(*results1), label="CF")
    plt.plot(*zip(*results2), label="MD")
    plt.ylabel("Execution Time (s)")
    plt.xlabel("RMSE")
    plt.legend()
    plt.show()
    
    return results1, results2 # return results for testing

# Uncomment to plot execution time vs RMSE. Takes around 35 minutes to run.
#results1, results2 = plot_execution_time_vs_rmse()